In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.callbacks import EarlyStopping
import random
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical


In [22]:
df = pd.read_excel('/Users/user/Downloads/lotto.xlsx')

In [32]:

# 데이터 준비
X = df.iloc[:-1, :6].values  # 마지막 회차 결과를 제외한 전체 회차 결과
y = df.iloc[1:, :6].values  # 첫 회차 결과를 제외한 전체 회차 결과


# 원-핫 인코딩
y_onehot = np.zeros((y.shape[0], y.shape[1], 46))
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        y_onehot[i, j, y[i, j]] = 1

# 데이터 스케일링 및 분할
input_scaler = MinMaxScaler()
X_scaled = input_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_onehot, test_size=0.2, random_state=42)

# LSTM 모델 생성 및 훈련
model = Sequential()
model.add(LSTM(32, activation='relu', input_shape=(6, 1), return_sequences=True))
model.add(TimeDistributed(Dense(46, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

early_stop = EarlyStopping(monitor='val_loss', patience=10)
model.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=16, callbacks=[early_stop])


def is_valid_combination(numbers):
    even_numbers = [num for num in numbers if num % 2 == 0]
    odd_numbers = [num for num in numbers if num % 2 == 1]
    first_half_numbers = [num for num in numbers if 1 <= num <= 22]
    second_half_numbers = [num for num in numbers if 23 <= num <= 45]

    if len(even_numbers) == len(numbers) or len(odd_numbers) == len(numbers):
        return False
    if len(first_half_numbers) == len(numbers) or len(second_half_numbers) == len(numbers):
        return False

    return True

def generate_recommendations(model, n_recommendations):
    recommendations = []
    previous_winners = [tuple(row) for row in y]
    
    last_x = X[-1].reshape(1, X[-1].shape[0], 1)
    predicted_probabilities = model.predict(last_x)

    while len(recommendations) < n_recommendations:
        # 각 번호별 확률을 기반으로 추천 번호 생성
        recommended_numbers = [np.random.choice(range(1, 46), p=probs[1:]) for probs in predicted_probabilities[0]]

        # 중복된 숫자가 있을 경우 새로운 숫자로 교체
        while len(set(recommended_numbers)) < len(recommended_numbers):
            for i, num in enumerate(recommended_numbers):
                if recommended_numbers.count(num) > 1:
                    num = np.random.choice(range(1, 46), p=predicted_probabilities[0, i, 1:])
                    recommended_numbers[i] = num

        # 이전 당첨 번호와 중복되지 않고 추천 목록에 없는 경우에만 추천 목록에 추가
        recommended_numbers_tuple = tuple(sorted(recommended_numbers))
        if recommended_numbers_tuple not in previous_winners and recommended_numbers_tuple not in recommendations:
            if is_valid_combination(recommended_numbers):
                recommendations.append(recommended_numbers)

    return recommendations



# 당첨 예측 번호 10세트 생성
recommended_numbers_list = generate_recommendations(model, 20)

# 예측 번호 출력
for i, numbers in enumerate(recommended_numbers_list):
    print(f"{i+1}: {numbers}")

    
    
def check_rank(y_true, y_pred, bonus_number):
    y_true = set(y_true)
    y_pred = set(y_pred[:6])

    correct_numbers = y_true.intersection(y_pred)
    count = len(correct_numbers)

    if count == 6:
        return 1
    elif count == 5 and bonus_number in y_pred:
        return 2
    elif count == 5:
        return 3
    elif count == 4:
        return 4
    elif count == 3:
        return 5
    else:
        return None
    
bonus_numbers = df.iloc[:-1, 6].values
# 각 추천 번호의 당첨 등수 확인
total_rank_counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
for numbers in recommended_numbers_list:
    for i in range(len(X) - 1):
        y_true = y[i]
        bonus_number = bonus_numbers[i]
        rank = check_rank(y_true, numbers, bonus_number)
        if rank is not None:
            total_rank_counts[rank] += 1

# 등수 출력 (꽝 제외)
print("전체 추천 번호의 전체 데이터에서의 당첨 횟수 합계 (꽝 제외):")
for rank, count in total_rank_counts.items():
    if count > 0:
        print(f"{rank}등: {count}회")




Epoch 1/50
48/48 [==============================] - 1s 7ms/step - loss: 3.8218 - accuracy: 0.0223 - val_loss: 3.8129 - val_accuracy: 0.0078
Epoch 2/50
48/48 [==============================] - 0s 3ms/step - loss: 3.7776 - accuracy: 0.0194 - val_loss: 3.7249 - val_accuracy: 0.0216
Epoch 3/50
48/48 [==============================] - 0s 3ms/step - loss: 3.6413 - accuracy: 0.0251 - val_loss: 3.6743 - val_accuracy: 0.0176
Epoch 4/50
48/48 [==============================] - 0s 3ms/step - loss: 3.5839 - accuracy: 0.0312 - val_loss: 3.6493 - val_accuracy: 0.0314
Epoch 5/50
48/48 [==============================] - 0s 3ms/step - loss: 3.5517 - accuracy: 0.0430 - val_loss: 3.6121 - val_accuracy: 0.0490
Epoch 6/50
48/48 [==============================] - 0s 3ms/step - loss: 3.5206 - accuracy: 0.0476 - val_loss: 3.5909 - val_accuracy: 0.0510
Epoch 7/50
48/48 [==============================] - 0s 3ms/step - loss: 3.4944 - accuracy: 0.0630 - val_loss: 3.5637 - val_accuracy: 0.0549
Epoch 8/50
48/48 [==

In [27]:
def calculate_number_probabilities(model):
    last_numbers = df.iloc[-1:, :6].values
    last_numbers_scaled = input_scaler.transform(last_numbers)
    last_numbers_scaled = last_numbers_scaled.reshape(last_numbers_scaled.shape[0], last_numbers_scaled.shape[1], 1)
    predicted_probabilities = model.predict(last_numbers_scaled)

    number_probabilities = {}
    for i, probabilities in enumerate(predicted_probabilities[0]):
        for num, prob in enumerate(probabilities[1:], start=1):
            if num not in number_probabilities:
                number_probabilities[num] = prob
            else:
                number_probabilities[num] += prob
    return number_probabilities

number_probabilities = calculate_number_probabilities(model)
sorted_number_probabilities = sorted(number_probabilities.items(), key=lambda x: x[1], reverse=True)

print("각 번호별 예측 당첨 확률 (내림차순):")
for num, prob in sorted_number_probabilities:
    print(f"{num}: {prob * 100:.2f}%")


1/1 [==============================] - 0s 18ms/step
각 번호별 예측 당첨 확률 (내림차순):
45: 16.47%
27: 16.39%
18: 15.96%
34: 15.37%
40: 15.10%
26: 14.89%
17: 14.72%
13: 14.65%
15: 14.64%
44: 14.54%
37: 14.50%
36: 14.48%
33: 14.46%
38: 14.33%
24: 14.32%
31: 14.28%
20: 13.98%
14: 13.91%
30: 13.77%
12: 13.66%
42: 13.65%
39: 13.65%
29: 13.57%
35: 13.49%
11: 13.41%
43: 13.14%
22: 13.09%
19: 12.87%
23: 12.75%
28: 12.73%
21: 12.58%
16: 12.52%
25: 12.45%
8: 12.29%
7: 12.01%
32: 11.94%
4: 11.73%
10: 11.71%
41: 11.65%
3: 11.48%
2: 11.29%
1: 10.93%
6: 10.54%
5: 10.48%
9: 9.53%


In [28]:
def check_duplicate_winners(y):
    sorted_winners = [sorted(row) for row in y]
    unique_winners = set(tuple(row) for row in sorted_winners)
    duplicate_exists = len(unique_winners) != len(y)
    return duplicate_exists

if check_duplicate_winners(y):
    print("중복된 당첨 번호가 있습니다.")
else:
    print("중복된 당첨 번호가 없습니다.")


중복된 당첨 번호가 없습니다.


In [29]:
def count_matches_3_or_more(data):
    count = 0
    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            common_numbers = set(data[i]).intersection(set(data[j]))
            if len(common_numbers) >= 3:
                count += 1
    return count


total_matches = count_matches_3_or_more(y)
print(f"전체 데이터 중에서 3개 이상 일치하는 조합 수: {total_matches}")


전체 데이터 중에서 3개 이상 일치하는 조합 수: 13313


In [30]:
import numpy as np

def count_even_odd(data):
    even_count = 0
    odd_count = 0
    for row in data:
        even_numbers = [num for num in row if num % 2 == 0]
        odd_numbers = [num for num in row if num % 2 == 1]
        if len(even_numbers) == len(row):
            even_count += 1
        if len(odd_numbers) == len(row):
            odd_count += 1
    return even_count, odd_count


def count_half_numbers(data):
    first_half_count = 0
    second_half_count = 0
    for row in data:
        first_half_numbers = [num for num in row if 1 <= num <= 22]
        second_half_numbers = [num for num in row if 23 <= num <= 45]
        if len(first_half_numbers) == len(row):
            first_half_count += 1
        if len(second_half_numbers) == len(row):
            second_half_count += 1
    return first_half_count, second_half_count


even_count, odd_count = count_even_odd(y)
first_half_count, second_half_count = count_half_numbers(y)

print(f"전체 데이터에서 짝수만 나온 횟수: {even_count}")
print(f"전체 데이터에서 홀수만 나온 횟수: {odd_count}")
print(f"전체 데이터에서 1-22 범위의 숫자만 나온 횟수: {first_half_count}")
print(f"전체 데이터에서 23-45 범위의 숫자만 나온 횟수: {second_half_count}")


전체 데이터에서 짝수만 나온 횟수: 13
전체 데이터에서 홀수만 나온 횟수: 18
전체 데이터에서 1-22 범위의 숫자만 나온 횟수: 14
전체 데이터에서 23-45 범위의 숫자만 나온 횟수: 12


In [39]:
import math

def nCr(n, r):
    return math.factorial(n) // (math.factorial(r) * math.factorial(n - r))

total_combinations = nCr(45, 6)
print(f"로또에서 가능한 조합의 총 개수: {total_combinations}")

# 45개 숫자 중에서 짝수와 홀수의 개수
even_numbers = 22
odd_numbers = 23

# 짝수만으로 이루어진 조합의 개수
even_combinations = nCr(even_numbers, 6)
print(f"짝수만으로 이루어진 조합의 개수: {even_combinations}")

# 홀수만으로 이루어진 조합의 개수
odd_combinations = nCr(odd_numbers, 6)
print(f"홀수만으로 이루어진 조합의 개수: {odd_combinations}")

# 1부터 22까지의 숫자 중에서 6개의 숫자를 선택하는 조합의 개수
combinations_1_22 = nCr(22, 6)
print(f"1부터 22까지의 숫자 중에서 6개의 숫자를 선택하는 모든 조합의 개수: {combinations_1_22}")

# 23부터 45까지의 숫자 중에서 6개의 숫자를 선택하는 조합의 개수
combinations_23_45 = nCr(23, 6)
print(f"23부터 45까지의 숫자 중에서 6개의 숫자를 선택하는 모든 조합의 개수: {combinations_23_45}")

total_combinations-2*(even_combinations+odd_combinations)


로또에서 가능한 조합의 총 개수: 8145060
짝수만으로 이루어진 조합의 개수: 74613
홀수만으로 이루어진 조합의 개수: 100947
1부터 22까지의 숫자 중에서 6개의 숫자를 선택하는 모든 조합의 개수: 74613
23부터 45까지의 숫자 중에서 6개의 숫자를 선택하는 모든 조합의 개수: 100947


7793940

In [40]:
def count_repeated_combinations(data, min_matches=3):
    count = 0
    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            common_numbers = set(data[i]).intersection(set(data[j]))
            if len(common_numbers) >= min_matches:
                count += 1
    return count

total_repeated_combinations = count_repeated_combinations(y, min_matches=3)
print(f"전체 데이터 중에서 3개 이상 일치하는 조합 수: {total_repeated_combinations}")


전체 데이터 중에서 3개 이상 일치하는 조합 수: 13313


In [44]:
from itertools import combinations

def count_3_or_more_combinations(data):
    all_combinations = set()
    for row in data:
        for comb in combinations(row, 3):
            all_combinations.add(tuple(sorted(comb)))
    return len(all_combinations)

total_combinations = count_3_or_more_combinations(y)
print(f"전체 데이터에서 고유한 3개 이상의 조합 수: {total_combinations}")


전체 데이터에서 고유한 3개 이상의 조합 수: 11101
